In [1]:
from gensim.models import KeyedVectors
import pandas as pd
import numpy as np
from recipe_loader import recipe_loader

# load ingredient vectors
wv = KeyedVectors.load("recipe2vec.wordvectors")

In [2]:
def recipe_vectorizer(recipe: str) -> np.ndarray:
    
    ingredient_vecs = np.array([wv[ingredient] for ingredient in recipe])
    
    recipe_vec = np.mean(ingredient_vecs,axis=0)
    
    return recipe_vec
    

Load the recipes:

In [3]:
recipes = recipe_loader()

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
recipes.columns

Index(['RecipeId', 'Name', 'AuthorId', 'AuthorName', 'CookTime', 'PrepTime',
       'TotalTime', 'DatePublished', 'Description', 'Images', 'RecipeCategory',
       'Keywords', 'RecipeIngredientQuantities', 'RecipeIngredientParts',
       'AggregatedRating', 'ReviewCount', 'Calories', 'FatContent',
       'SaturatedFatContent', 'CholesterolContent', 'SodiumContent',
       'CarbohydrateContent', 'FiberContent', 'SugarContent', 'ProteinContent',
       'RecipeServings', 'RecipeYield', 'RecipeInstructions', 'Ingredients'],
      dtype='object')

In [ ]:
recipe_vectors = recipes["Ingredients"].apply(recipe_vectorizer)

In [ ]:
from annoy import AnnoyIndex

vec_size = 100

index = AnnoyIndex(vec_size, "angular")

for i, vec in enumerate(recipe_vectors.values):
    index.add_item(i, vec)
    
index.build(10)
index.save(f"recipe2vecIndex.ann")

True

In [ ]:
recipes.head()

NameError: name 'recipes' is not defined

In [ ]:
indexes = index.get_nns_by_item(0, 10)

recipes.iloc[indexes]

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions,Ingredients
0,38,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09T21:46:00Z,Make and share this Low-Fat Berry Blue Frozen ...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,8.0,29.8,37.1,3.6,30.2,3.2,4.0,NaN,"c(""Toss 2 cups berries with sugar."", ""Let stan...","(blueberries, granulated sugar, vanilla yogurt..."
430950,446923,Easy Gluten Free Cereal,1747478,Attainable Health,NaN,PT2M,PT2M,2011-01-20T14:42:00Z,I wanted cereal one morning but didn't have an...,"""https://img.sndimg.com/food/image/upload/w_55...",...,8.0,29.4,15.3,3.8,8.8,8.4,1.0,NaN,"c(""Place walnuts in a ziploc bag then use a he...","(walnuts, vanilla yogurt, blueberries)"
165940,173710,Apple Blueberry Smoothie,37449,Sharon123,NaN,PT5M,PT5M,2006-06-19T16:30:00Z,This is a British Columbian recipe from Canada...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,24.5,88.5,22.5,2.4,15.9,6.7,2.0,NaN,"""Place all ingredients in blender and blend un...","(apple, vanilla yogurt, milk, blueberries, sugar)"
418340,433731,Beautiful Berry Pie,883095,Chef mariajane,PT45M,PT0S,PT45M,2010-07-28T02:43:00Z,Make and share this Beautiful Berry Pie recipe...,character(0),...,0.0,138.1,51.4,4.2,29.9,2.9,6.0,NaN,"c(""Preheat oven to 425°F."", ""Mix sugar, flour ...","(kiwi fruits, blueberries, strawberries, granu..."
386548,400464,Yogurt Berry Parfaits,463435,Chef Buggsy Mate,PT5M,PT20M,PT25M,2009-11-20T15:34:00Z,A tasty treat when fresh berries are in season...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,15.8,233.6,47.7,5.2,31.2,7.0,4.0,4 parfaits,"c(""In a small bowl, stir together the yogurt a...","(plain yogurt, prepared vanilla pudding, rolle..."
62518,66945,Frosty Fruity Smoothie,32772,Audrey M,NaN,PT5M,PT5M,2003-07-16T20:04:00Z,A cool and refreshing smoothie. To make it low...,character(0),...,31.9,119.0,69.8,4.7,57.0,9.5,2.0,NaN,"c(""Combine blueberries or huckleberries, peach...","(blueberries, huckleberries, milk, vanilla yog..."
247701,257791,The Berry Best Berries! Topping,408968,Krystalin,PT55M,PT0S,PT55M,2007-10-08T22:35:00Z,I found this recipe in a local supermarket pam...,character(0),...,0.0,2.2,13.9,2.9,9.8,0.7,NaN,NaN,"c(""In small pot, combine sugar, zest, and wate...","(sugar, lemon zest, orange, zest of, water, le..."
164419,172125,Cool Blueberry Soup,300480,dianahirkala,PT1H,PT5M,PT1H5M,2006-06-09T18:30:00Z,Make and share this Cool Blueberry Soup recipe...,character(0),...,0.0,3.1,31.9,2.9,26.7,1.3,NaN,3 cups,"c(""Combine pineapple and lemon juices, water a...","(water, lemon juice, vanilla extract, blueberr..."
245000,254992,French Blueberry Tart,133174,PaulaG,PT24H,PT20M,PT24H20M,2007-09-24T20:38:00Z,Yogurt cheese is a wonderful way to reduce cal...,"""https://img.sndimg.com/food/image/upload/w_55...",...,3.3,45.1,31.0,1.3,27.3,3.7,NaN,NaN,"c(""Place a yogurt strainer or a mesh strainer ...","(low-fat vanilla yogurt, powdered sugar, blueb..."
313051,325021,Blueberry Ice Cream,210188,Nana Lee,NaN,PT20M,PT20M,2008-09-13T13:49:00Z,Found on ricecooker site. Sounds delisc! Pos...,character(0),...,633.6,393.7,426.0,14.3,364.0,30.6,NaN,1 quart,"c(""In a 3-quart saucepan combines blueberries,...","(blueberries, sugar, vanilla extract)"


In [ ]:
recipes.columns

NameError: name 'recipes' is not defined